In [ ]:
#1 - a perceptron is a neural network with a single layer of input linear neurons followed by an
#output unit based on the sign(x) function

#2 - multilayer perceptron: adds at least one non-linear layer between the input and output which 
#leads to a highly non-linear combination, parametrized with a larger number of variables

#3 - a hard margin is very rigid in classification and tends to overfit because it tries to work extremely 
#well in the training set. It also addresses the generalization problem of perceptrons

#4 - D

#5 - D

In [1]:
pip install tensorflow

     |████████████████████████████████| 458.3 MB 17 kB/s s eta 0:00:01    |█████▎                          | 76.0 MB 65.6 MB/s eta 0:00:06     |███████████████████████████     | 387.5 MB 73.0 MB/s eta 0:00:01     |█████████████████████████████▍  | 420.2 MB 73.0 MB/s eta 0:00:01     |██████████████████████████████▍ | 435.3 MB 25.5 MB/s eta 0:00:01     |██████████████████████████████▊ | 439.3 MB 25.5 MB/s eta 0:00:01     |██████████████████████████████▊ | 440.8 MB 25.5 MB/s eta 0:00:01
     |████████████████████████████████| 65 kB 7.4 MB/s  eta 0:00:01
     |████████████████████████████████| 5.6 MB 50.9 MB/s eta 0:00:01
     |████████████████████████████████| 3.9 MB 44.9 MB/s eta 0:00:01
     |████████████████████████████████| 462 kB 19.9 MB/s eta 0:00:01
     |████████████████████████████████| 131 kB 71.6 MB/s eta 0:00:01
     |████████████████████████████████| 1.3 MB 42.5 MB/s eta 0:00:01
     |████████████████████████████████| 42 kB 250 kB/s  eta 0:00:01
     |████████████████████████

In [6]:
import boto3 
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression, Ridge, RidgeCV, Lasso, LassoCV
from sklearn.linear_model import LogisticRegression, Perceptron
from sklearn.metrics import accuracy_score 
from sklearn.preprocessing import MinMaxScaler
import tensorflow as tf
from sklearn.svm import SVC

#defining bucket
s3 = boto3.resource('s3')
bucket_name = 'data445-ashlyn-bucket' 
bucket = s3.Bucket(bucket_name)

#defining csv file
file_key = 'framingham.csv'

bucket_object = bucket.Object(file_key)
file_object = bucket_object.get()
file_content_stream = file_object.get('Body') 

#reading csv file
heart = pd.read_csv(file_content_stream)
heart.head()

,male,age,education,currentSmoker,cigsPerDay,BPMeds,prevalentStroke,prevalentHyp,diabetes,totChol,sysBP,diaBP,BMI,heartRate,glucose,TenYearCHD
0,1,39,4.0,0,0.0,0.0,0,0,0,195.0,106.0,70.0,26.97,80.0,77.0,0
1,0,46,2.0,0,0.0,0.0,0,0,0,250.0,121.0,81.0,28.73,95.0,76.0,0
2,1,48,1.0,1,20.0,0.0,0,0,0,245.0,127.5,80.0,25.34,75.0,70.0,0
3,0,61,3.0,1,30.0,0.0,0,1,0,225.0,150.0,95.0,28.58,65.0,103.0,1
4,0,46,3.0,1,23.0,0.0,0,0,0,285.0,130.0,84.0,23.10,85.0,85.0,0


In [3]:
#removing missing values
heart = heart.dropna()
heart.head()
heart = heart.reset_index(drop = True)

In [4]:
#defining the input and target variables
X = heart[['age', 'currentSmoker', 'totChol', 'BMI', 'heartRate']]
Y = heart['TenYearCHD']

## splitting the data
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.2)

In [5]:
## Transforming input data to 0-1
scaler = MinMaxScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.fit_transform(X_test)

In [9]:
md1 = tf.keras.models.Sequential([
    tf.keras.layers.Dense(4, input_dim = 5, activation = 'tanh'),
    tf.keras.layers.Dense(2, activation = 'softmax')
])

md1.compile(optimizer = 'sgd', metrics = ['accuracy'], loss = 'categorical_crossentropy')
md1.fit(X_train, tf.keras.utils.to_categorical(Y_train, num_classes = 2), epochs = 100, batch_size = 500, validation_data = (X_test, tf.keras.utils.to_categorical(Y_test, num_classes = 2)), verbose = 0)

In [30]:
from sklearn.metrics import recall_score

#reporting recall score
md1_preds = md1.predict(X_test)[:,1]
md1_preds = np.where(md1_preds < 0.15, 0, 1)

recall_score(Y_test, md1_preds)

1.0

In [14]:
md2 = tf.keras.models.Sequential([
    tf.keras.layers.Dense(4, input_dim = 5, activation = 'relu'),
    tf.keras.layers.Dense(2, activation = 'softmax')
])

md2.compile(optimizer = 'sgd', metrics = ['accuracy'], loss = 'categorical_crossentropy')
md2.fit(X_train, tf.keras.utils.to_categorical(Y_train, num_classes = 2), epochs = 100, batch_size = 500, validation_data = (X_test, tf.keras.utils.to_categorical(Y_test, num_classes = 2)), verbose = 0)

In [19]:
#reporting recall score
md2_preds = md2.predict(X_test)[:,1]
md2_preds = np.where(md2_preds < 0.15, 0, 1)

recall_score(Y_test, md2_preds)

1.0

In [24]:
#building svm with kernel = 'rbf'
md3 = SVC(kernel = 'rbf', probability = True).fit(X_train, Y_train)

md3_preds = md3.predict_proba(X_test)[:,1]
md3_preds = np.where(md3_preds < 0.15, 0, 1)

recall_score(Y_test, md3_preds)

0.6074766355140186

In [25]:
#building svm with kernel = 'poly'
md4 = SVC(kernel = 'rbf', probability = True).fit(X_train, Y_train)

md4_preds = md4.predict_proba(X_test)[:,1]
md4_preds = np.where(md4_preds < 0.15, 0, 1)

recall_score(Y_test, md4_preds)

0.5981308411214953

In [31]:
#defining the input and target variables
X = heart[['age', 'currentSmoker', 'totChol', 'BMI', 'heartRate']]
Y = heart['TenYearCHD']

## splitting the data
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.2)

for i in range(0, 100):
    scaler = MinMaxScaler()
    X_train = scaler.fit_transform(X_train)
    X_test = scaler.fit_transform(X_test)
    
    #model 1
    md1 = tf.keras.models.Sequential([
    tf.keras.layers.Dense(4, input_dim = 5, activation = 'tanh'),
    tf.keras.layers.Dense(2, activation = 'softmax')
    ])

    md1.compile(optimizer = 'sgd', metrics = ['accuracy'], loss = 'categorical_crossentropy')
    md1.fit(X_train, tf.keras.utils.to_categorical(Y_train, num_classes = 2), epochs = 100, batch_size = 500, validation_data = (X_test, tf.keras.utils.to_categorical(Y_test, num_classes = 2)), verbose = 0)
    
    md1_preds = md1.predict(X_test)[:,1]
    md1_preds = np.where(md1_preds < 0.15, 0, 1)
    model1 = recall_score(Y_test, md1_preds)

    md2 = tf.keras.models.Sequential([
    tf.keras.layers.Dense(4, input_dim = 5, activation = 'relu'),
    tf.keras.layers.Dense(2, activation = 'softmax')
    ])
    
    #model 2
    md2.compile(optimizer = 'sgd', metrics = ['accuracy'], loss = 'categorical_crossentropy')
    md2.fit(X_train, tf.keras.utils.to_categorical(Y_train, num_classes = 2), epochs = 100, batch_size = 500, validation_data = (X_test, tf.keras.utils.to_categorical(Y_test, num_classes = 2)), verbose = 0)
    
    #reporting recall score
    md2_preds = md2.predict(X_test)[:,1]
    md2_preds = np.where(md2_preds < 0.15, 0, 1)
    model2 = recall_score(Y_test, md2_preds)
    
    #model 3

    md3 = SVC(kernel = 'rbf', probability = True).fit(X_train, Y_train)

    md3_preds = md3.predict_proba(X_test)[:,1]
    md3_preds = np.where(md3_preds < 0.15, 0, 1)

    model3 = recall_score(Y_test, md3_preds)
    
    #model 4
    md4 = SVC(kernel = 'rbf', probability = True).fit(X_train, Y_train)

    md4_preds = md4.predict_proba(X_test)[:,1]
    md4_preds = np.where(md4_preds < 0.15, 0, 1)

    model4 = recall_score(Y_test, md4_preds)    

ValueError: Input contains NaN, infinity or a value too large for dtype('float64').